In [5]:
from bs4 import BeautifulSoup
import requests as requests
import time
import re
from datetime import datetime
import pandas as pd
import numpy as np

In [3]:
## Key variables valid for all the chuncks
timestamp_pattern = re.compile(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{3}Z)')
original_format = "%Y-%m-%dT%H:%M:%S.%fZ"
desired_format = "%Y-%m-%d %H:%M:%S"


## Test chunck
NB: need to solve the problem of some items not having a title (these are the comments). At the current time, I solved the problem with a `try - except`.

In [4]:
titles = []

start = time.time()

for i in range(1,11):
    
    try:
        url_posts = 'https://stacker.news/items/%d' % (i)
        response = requests.get(url_posts)
        soup = BeautifulSoup(response.text, 'html.parser')
    
        title = soup.find_all('a', class_='item_title__FH7AS text-reset me-2')
        
        for e in title:
            titles.append(e.get_text())
        
    except:
        continue

end = time.time()

print("The time of execution of above program is :",
      (end-start), "s")

## Collect post header with all the related data

In [47]:
url_posts = 'https://stacker.news/items/1'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
banner = soup.find('div', class_='item_other__MjgP3')

#post_stack = banner.find('span').text

banner_data = []

for i in banner.find_all('span'):
    banner_data.append(i.text)
    
#n_comments = soup.find('a', class_='text-reset position-relative').get_text()
#nym_creator = banner.find('a', ) 

print([titolo, banner_data])


['El Salvador Makes Bitcoin Legal Tender', ['603 sats', ' \\ ', '4 boost', ' \\ ', ' \\ ', '@k00b  11 Jun 2021', ' ', '', ' ', 'bitcoin', '']]


## Collect timestamp of post creation
The following code scans the parsed html response looking for an `<a>` tag that has an attribute `title` matching the timestamp format in the website.
Then it extracts it and turns it into a datetime object. 

In [20]:
url_posts = 'https://stacker.news/items/1'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

timestamp_element = soup.find('a', attrs={'title': timestamp_pattern})
timestamp_match = timestamp_pattern.search(timestamp_element['title']).group(1)

timestamp_datetime = datetime.strptime(timestamp_match, original_format)

print(timestamp_datetime)


2021-06-11 19:26:02.662000


# Scraping Items of stacker.news

In [7]:
df = pd.DataFrame(columns=['titolo', 'numero commenti', 'boost', 'sats', 'betha'])
n_posts = range(1,20)

for number in n_posts:
    print("Doing", number)
    url_posts = f'https://stacker.news/items/{number}'
    response = requests.get(url_posts)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        contenuto=soup.find('div', class_="item_fullItemContainer__ZAYtZ").get_text()
    except:
        continue
    try:
        titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
    except:
        continue
    try:
        banner = soup.find('div', class_='item_other__MjgP3')
    except:
        continue
    try:
        commento = soup.find('a', class_='text-reset position-relative').get_text()
    except:        
        continue

    banner_data = []
    for i in banner.find_all('span'):
        banner_data.append(i.text)
    index=number-1
    for b in banner_data:
        if "boost" in b:
            df.at[index, 'boost'] = b
        if "sats" in b:
            df.at[index, 'sats'] = b
        if "@" in b:
            df.at[index, 'betha'] = b

    df.at[index, 'titolo'] = titolo
    df.at[index, 'numero commenti'] = commento
    df.at[index, 'contenuto'] = contenuto

df['author'] = df['betha'].str.extract(r'@(\w+)')
df['data'] = df['betha'].str.extract(r'(\d+\s\w+\s\d+)')
df.drop('betha', axis=1, inplace=True)

df


Doing 1


/tmp/ipykernel_2556/2015901663.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'contenuto'] = contenuto


Doing 2
Doing 3
Doing 4
Doing 5
Doing 6
Doing 7
Doing 8
Doing 9
Doing 10
Doing 11
Doing 12
Doing 13
Doing 14
Doing 15
Doing 16
Doing 17
Doing 18
Doing 19


,titolo,numero commenti,boost,sats,contenuto,author,data
0,El Salvador Makes Bitcoin Legal Tender,13 comments,4 boost,603 sats,,k00b,11 Jun 2021
1,stacker news soft launch AMA,30 comments,NaN,2 sats,Figured I'd make a place to raise questions ab...,k00b,11 Jun 2021
7,The Lightning Conference 2019: Alex Bosworth,2 comments,NaN,0 sats,,k00b,11 Jun 2021


# Scraping user profiles
Users profiles are scraped starting from the list of users extracted by scraping all the items (posts+comments)
The link to get the user profile is `https://stacker.news/$username$` 

In [106]:
## Test user - the creator of stacker.news
# 
name = 'ppp'

url_posts = f'https://stacker.news/{name}'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

if str(response)=='<Response [404]>':
    print(None)


# # This could became a function
# nym_tot_stacked = soup.find('div', class_='mb-2 ms-0 ms-sm-1 user-header_username__bqOV1 text-success').get_text()
# regex_sats = r'[^\d]+'
# nym_tot_stacked = int(re.sub(regex_sats, '', nym_tot_stacked))

# # Nym address
# nym_lnaddress = soup.find('button', class_='fw-bold ms-0 btn btn-primary', type='button').get_text()

# # Stacking since -> first user appearance (measured by the first item he/she created)
# # We need to decide the datatype for this index -> string is better
# nym_first_item = soup.find('a', class_='ms-1').get_text()[1:]

# # Longest cowboy hat streak (README.md for cowboy-hat meaning)
# # This could became a function
# nym_ch_streak = soup.find_all('small', class_='text-muted d-flex-inline')[1].text
# regex_ch = r'[^\d]+'
# nym_ch_streak = int(re.sub(regex_ch, '', nym_ch_streak))

# # Total number of Items created by the user
# nym_tot_items = soup.find('a', class_='nav-link', href=f'/{name}/all').get_text()
# regex_nym_items = r' items\b'
# nym_tot_items = int(re.sub(regex_nym_items, '', nym_tot_items))

#print([name, nym_tot_stacked, nym_lnaddress, nym_first_item, nym_ch_streak, nym_tot_items])


None


## Modularize the code - user profile scraping
Define functions to modularize and simplify the user profile scraping

In [126]:
## Fixed value to be returned for missing values or request errors
NA = np.nan

## Function that returns the webpage where the user profile is
def get_profile(name):
    
    try:
        # Provided a string returns a bs4.BeautifulSoup object
        url_posts = f'https://stacker.news/{name}'
        response = requests.get(url_posts)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    except:
        return NA
        

## Function that returns the total amount of sats stacked by the user
def get_total_stacked(name):
    
    # Provided a string returns an integer
    page = get_profile(name)
    try:
        nym_tot_stacked = page.find('div', class_='mb-2 ms-0 ms-sm-1 user-header_username__bqOV1 text-success').get_text()
        regex_sats = r'[^\d]+'
        nym_tot_stacked = int(re.sub(regex_sats, '', nym_tot_stacked))
        return nym_tot_stacked 
    except:
        return NA


## Function that returns the first user appearance (measured by the code of the first item created)
def get_stacking_since(name):
    
    # Provided a string returns a string
    page = get_profile(name)
    try:
        nym_first_item = page.find('a', class_='ms-1').get_text()[1:]
        return nym_first_item
    except:
        return NA


## Function that returns the longest cowboy-hat streak
## If a user tip 100 sats (or more) in a day it gets a cowboy hat for that day
def get_cowboy_streak(name):
    
    # Provided a string returns an integer
    page = get_profile(name)
    try:
        nym_ch_streak = page.find_all('small', class_='text-muted d-flex-inline')[1].text
        regex_ch = r'[^\d]+'
        nym_ch_streak = int(re.sub(regex_ch, '', nym_ch_streak))
        return nym_ch_streak
    except:
        return NA


## Function that returns the total number of Items created by the user
def get_total_items(name):
    
    # Provided a string returns an integer
    page = get_profile(name)
    try:
        nym_tot_items = page.find('a', class_='nav-link', href=f'/{name}/all').get_text()
        regex_nym_items = r' items\b'
        nym_tot_items = int(re.sub(regex_nym_items, '', nym_tot_items))
        return nym_tot_items
    except:
        return NA


### Testing functions for code modularization
There are a few corner cases to test

In [131]:
import unittest
import numpy as np
from bs4 import BeautifulSoup

# # Import your functions here
# from your_module import (
#     get_profile,
#     get_total_stacked,
#     get_stacking_since,
#     get_cowboy_streak,
#     get_total_items,
# )

NA = np.nan

# Create a test class
class TestUserProfileFunctions(unittest.TestCase):

    def test_get_profile(self):
        # Test a valid profile
        profile = get_profile("valid_username")
        self.assertIsInstance(profile, BeautifulSoup)

        # Test an invalid profile
        invalid_profile = get_profile("invalid_username")
        self.assertEqual(invalid_profile, NA)

    def test_get_total_stacked(self):
        # Test a valid profile with total stacked
        total_stacked = get_total_stacked("valid_username_with_total_stacked")
        self.assertIsInstance(total_stacked, int)

        # Test a valid profile without total stacked
        total_stacked_missing = get_total_stacked("valid_username_without_total_stacked")
        self.assertEqual(total_stacked_missing, NA)

        # Test an invalid profile
        invalid_profile = get_total_stacked("invalid_username")
        self.assertEqual(invalid_profile, NA)

    def test_get_stacking_since(self):
        # Test a valid profile with stacking since date
        stacking_since = get_stacking_since("valid_username_with_stacking_since")
        self.assertIsInstance(stacking_since, str)

        # Test a valid profile without stacking since date
        stacking_since_missing = get_stacking_since("valid_username_without_stacking_since")
        self.assertEqual(stacking_since_missing, NA)

        # Test an invalid profile
        invalid_profile = get_stacking_since("invalid_username")
        self.assertEqual(invalid_profile, NA)

    def test_get_cowboy_streak(self):
        # Test a valid profile with cowboy streak
        cowboy_streak = get_cowboy_streak("valid_username_with_cowboy_streak")
        self.assertIsInstance(cowboy_streak, int)

        # Test a valid profile without cowboy streak
        cowboy_streak_missing = get_cowboy_streak("valid_username_without_cowboy_streak")
        self.assertEqual(cowboy_streak_missing, NA)

        # Test an invalid profile
        invalid_profile = get_cowboy_streak("invalid_username")
        self.assertEqual(invalid_profile, NA)

    def test_get_total_items(self):
        # Test a valid profile with total items
        total_items = get_total_items("valid_username_with_total_items")
        self.assertIsInstance(total_items, int)

        # Test a valid profile without total items
        total_items_missing = get_total_items("valid_username_without_total_items")
        self.assertEqual(total_items_missing, NA)

        # Test an invalid profile
        invalid_profile = get_total_items("invalid_username")
        self.assertEqual(invalid_profile, NA)


if __name__ == "__main__":
    unittest.main()


E
ERROR: /home/filippo/ (unittest.loader._FailedTest./home/filippo/)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/filippo/'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/home/filippo/repositories/stacker-news/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
